<a href="https://colab.research.google.com/github/vicssb/lab_asset_forecast/blob/main/asset_forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Setup

###Install the Python SDK

Install the Python SDK for the Gemini API, is contained in the google-generativeai package. Install the dependency using pip:

In [82]:
pip install -q -U google-generativeai


###Import packages

Import the necessary packages.

In [83]:
import pathlib
import textwrap
import numpy as np
import pandas as pd
import google.generativeai as genai
from pathlib import Path
import hashlib

from google.colab import userdata
from IPython.display import display
from IPython.display import Markdown
from sklearn.linear_model import LinearRegression


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

### Setup your API key

Before you can use the Gemini API, you must first obtain an API key. If you don't already have one, create a key with one click in Google AI Studio.

<a class="button button-primary" href="https://makersuite.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Get an API key</a>

In [84]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.

api_key = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key = api_key)

### Set up the model

In [85]:


generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 1,
  "max_output_tokens": 1024,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              safety_settings=safety_settings)



###Prompt

In [90]:
prompt_parts = [
  "Extraia as seguinte informações das imagens: hora (na parte inferior da imagem) e valor (na parte direita da imagem). crie uma tabela com os resultados.\n\n",
  genai.upload_file("/content/imgs/805.png"),
  genai.upload_file("/content/imgs/810.png"),
  genai.upload_file("/content/imgs/821.png"),
  genai.upload_file("/content/imgs/825.png"),
  genai.upload_file("/content/imgs/830.png"),
  genai.upload_file("/content/imgs/945.png"),
  genai.upload_file("/content/imgs/114.png"),
  genai.upload_file("/content/imgs/1030.png"),
  genai.upload_file("/content/imgs/1100.png"),
  genai.upload_file("/content/imgs/1130.png"),
  "salve a tabela em um arquivo /content/imgs/tab.csv",
]
#"/content/drive/MyDrive/Colab Notebooks/iqoption/945.png"

#prompt_parts.append = input('img: ')
#prompt_parts.append = genai.upload_file(input('img: '))

response = model.generate_content(prompt_parts)
print(response.text)
#df = pd.DataFrame(response.text)
#df.to_csv('/content/imgs/tab.csv', index=False)


## Tabela de dados das imagens

| Imagem | Hora | Valor |
|---|---|---|
| 1 | 09:00:00 | 28.99500 |
| 2 | 08:10:05 (UTC) | 28.91500 |
| 3 | 08:27:09 | 28.87500 |
| 4 |  | 28.99500 |
| 5 | 08:35:00 | 28.99500 |
| 6 | 10:15:00 | 28.99900 |
| 7 | 10:45:00 | 29.03500 |
| 8 | 11:00:00 | 29.06500 |
| 9 | 11:30:00 | 29.16500 |
| 10 | 12:00:00 | 29.08500 |


**Arquivo tab.csv salvo em /content/imgs/tab.csv** 

**Observação:** A imagem 4 não possui a informação de hora visível. 



###Predição

In [97]:
# Criar DataFrame com os dados
data = {'Hora': ['09:00:00', '08:10:05', '11:08:27', '08:00:00', '08:35:00', '10:15:00', '10:45:00', '10:45:00', '11:30:00'],
        'Valor': [28.99500, 28.91500, 28.87500, 28.99500, 28.89500, 28.99900, 29.03500, 29.06500, 29.16500]}
df = pd.DataFrame(data)

# Converter a hora para minutos
df['Minutos'] = df['Hora'].apply(lambda x: int(x[:2]) * 60 + int(x[3:5]) + int(x[6:]) / 60)

# Criar modelo de regressão linear
modelo = LinearRegression()
modelo.fit(df[['Minutos']], df['Valor'])

# Prever o valor para 12:00 (720 minutos)
projecao_12h = modelo.predict([[720]])

print(f"Projeção do valor às 12:00: {projecao_12h[0]:.5f}")

Projeção do valor às 12:00: 29.06616


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
